# Projeto 1 - Ciência dos Dados

Nome: Alison Araujo

Nome: Gabrielly Carneiro

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [574]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('word_tokenize')
stopwordsdic = stopwords.words('portuguese')

from spacy import load
nlp = load('pt_core_news_sm')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading word_tokenize: Package 'word_tokenize' not
[nltk_data]     found in index


In [575]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\gabri\OneDrive - Insper - Institudo de Ensino e Pesquisa\Projeto 1\C.DadosP1


Carregando a base de dados com os tweets classificados manualmente:

In [576]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)

,Mensagem,Acionável/Direcionável/Não Acionável
0,"Para quem gosta de poemas simples, esse é o li...",N
1,Quando vi o lançamento pensei que finalmente p...,D
2,"É incrível como esses escritores e ""intelectua...",N
3,"Se desse pra devolver eu devolvia, nao é por e...",N
4,"Futilidade, inutilidade, desperdício de papel,...",N


In [577]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,Acionável/Direcionável/Não Acionável
0,"O livro é prolixo, redundante, doentio. Sou su...",N
1,"Bom livro e história envolvente. Porém, o leit...",D
2,"Fala tudo que todos já sabem, sem falar que nã...",N
3,Ganhei na compra do Kindle. Não é estilo de li...,N
4,"Custa crer que um livro tão medíocre, embora m...",N


In [578]:
pequena = pd.read_excel('planilha_teste.xlsx')
pequena.head(15)

,Mensagem,Acionável/Direcionável/Não Acionável
0,"Para quem gosta de poemas simples, esse é o li...",N
1,Quando vi o lançamento pensei que finalmente p...,D
2,"É incrível como esses escritores e ""intelectua...",N
3,"Se desse pra devolver eu devolvia, nao é por e...",N
4,"Futilidade, inutilidade, desperdício de papel,...",N
5,"O autor tenta mostrar outro lado do livro, bus...",N
6,"Começou boa istoria, no meio um pouco enrolado...",N
7,"Bem, eu nem sei por onde começar. A história é...",N
8,"Menos do menos, não mais do mesmo. As dicas da...",N
9,Meu box veio todo detonado. Fiquei triste porq...,A


___
## Classificador automático


Na primeira etapa, de classificação manual, consideramos três targets para os reviews: Acionável, Direcionável e Não Acionável. 

- Acionável: para ser considerado "acionável" ("A") o review deve ser passível de alguma ação pela Amazon, ou seja, o review deve ser sobre entrega, estado do produto, contato com o suporte, etc. 
- Direcionável: para o target "direcionável" ("D") foram considerados comentários relativos à editora, como qualidade do material do livro, preço do livro e do e-book, tradução e edição. 
- Não Acionáveis: por fim, os não acionáveis ("NA") eram comentários relativos ao autor, ao apreço pelo conteúdo do livro, ou comentários irrelavantes.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Funções úteis:

In [579]:
#Criar uma função que transforma as frases da planilha em um texto só 
    #(Será útil para criar o dicionário com as palavras)

def transforma_em_string(coluna):
    texto = ''
    for linha in coluna:
        texto += linha + ' '
    return texto    

In [580]:
#Criar uma função que limpa todas as pontuações
#recebe um texto
#vou utilizar ela na função limpa tudo
def cleanup(text):
    punctuation = r'[´"\'!-.:?;$,/~^_=+*&¨%$#@|\{}()[\]]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [581]:
#Cria uma função que limpa os espaços duplicados
#vou utilizar ela na função limpa tudo
def limpa_espaco(text):
    punctuation = r'[\n]'  # Adicione os caracteres desejados aqui
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed



In [582]:
#Criando uma função para remover emoji
#vou utilizar ela na função limpa tudo
def remove_emoji(text):
    text_without_emojis = unidecode(text)
    return text_without_emojis

In [583]:
#Criando a função de stopwords
#vou utilizar ela na função limpa tudo
#vai receber o texto limpo pelas outras funções
def stopwords(texto):
    palavras = word_tokenize(texto, language='portuguese') # Tokenize é analisar palavras individualmente, basicamente
    palavras_sem_stopword = []
    for palavra in palavras:
        if palavra not in stopwordsdic:
            palavras_sem_stopword.append(palavra)
    # Reúna as palavras sem stopwords em uma string novamente
    texto_sem_stopword = ' '.join(palavras_sem_stopword)
    return texto_sem_stopword

In [584]:
#Criando a função de lematização
#vou utilizar ela na função limpa tudo
#vai receber o texto limpo pelas outras funções, incluindo limpeza de stopwords

def lemmat(texto):
    doc = nlp(texto)
    lemmat_radicais = []
    for radicais in doc:
        lemmat_radicais.append(radicais.lemma_)
    texto_lemmat = ' '.join(lemmat_radicais)    
    return texto_lemmat

In [585]:
# Cria uma função que reúna as funções de limpeza
def limpa_tudo(mensagem):
    #Aplicando a função de limpeza de pontuação
    texto = cleanup(mensagem)
    #Deixando tudo em letra minúscula
    texto = texto.lower()
    #Removendo emoji
    texto = remove_emoji(texto)
    #Aplicando a função de limpeza de espaço
    texto = limpa_espaco(texto)
    #Removendo stopwords
    texto = stopwords(texto)
    #Realiza lemmatização
    texto = lemmat(texto)          # removi pois demora muito para classificar 
    return texto

In [586]:
#Cria uma função que limpa as linhas da planilha e adiciona uma coluna com as mensagens limpas à planilha
def mensagem_limpa(planilha):   #recebe a planilha e cria uma nova planilha com a coluna de mensagem limpa com as frases limpas
    planilha_limpa = planilha.copy()
    planilha_limpa['Mensagem Limpa'] = [limpa_tudo(x) for x in list(planilha['Mensagem'])]
    return planilha_limpa    #retorna a planilha modificada

planilha_limpa = mensagem_limpa(train)
planilha_limpa


,Mensagem,Acionável/Direcionável/Não Acionável,Mensagem Limpa
0,"Para quem gosta de poemas simples, esse é o li...",N,gostar poema simples livro ideal autora fazer ...
1,Quando vi o lançamento pensei que finalmente p...,D,ver lancamento pensar finalmente poder livrar ...
2,"É incrível como esses escritores e ""intelectua...",N,incrivel escritor intelectual colocar cristian...
3,"Se desse pra devolver eu devolvia, nao é por e...",N,de esse pra devolver devolvia nao nao pensar i...
4,"Futilidade, inutilidade, desperdício de papel,...",N,futilidade inutilidade desperdicio papel tempo...
...,...,...,...
495,"Mal comecei a ler e já parei, é um livro que f...",N,mal começar ler ja parei livro falar sobre teo...
496,Comprei o livro baseado na qtd de avaliações ...,N,compr livro basear qtd avaliacoe acima media r...
497,Comprei esse livro (todo seu ) paguei a fatura...,A,comprar livro todo paguei fatura ate hoje livr...
498,meu box chegou com alguma bolhas nas capas de ...,A,box chegar algum bolha capa algum livro orelha...


In [587]:
#Cria vocabulário de tudo
def cria_vocabulario(coluna_mensagem_limpa_da_planilha_limpa):                       #recebe uma coluna da planilha
    lista_palavras = transforma_em_string(coluna_mensagem_limpa_da_planilha_limpa)
    lista_palavras = lista_palavras.split()
    return lista_palavras     #devolve uma lista com as palavras separadas

planilha_limpa = mensagem_limpa(train)
lista_palavras_planilha_train = planilha_limpa["Mensagem Limpa"]
print(cria_vocabulario(lista_palavras_planilha_train))

print(planilha_limpa["Mensagem Limpa"])


['gostar', 'poema', 'simples', 'livro', 'ideal', 'autora', 'fazer', 'refletir', 'sobre', 'algum', 'elemento', 'comum', 'vida', 'mulher', 'jeito', 'bem', 'levar', 'ver', 'lancamento', 'pensar', 'finalmente', 'poder', 'livrar', 'ebook', 'distribuir', 'net', 'graca', 'ver', 'precar', 'mudei', 'ideia', 'esperar', 'ninguem', 'compre', 'querer', 'livro', 'digital', 'democratico', 'precos', 'acessivel', 'nao', 'trampolim', 'riqueza', 'livro', 'digital', 'nao', 'utilizar', 'tinta', 'papel', 'nao', 'precisar', 'distribuicao', 'fisico', 'dispensar', 'muito', 'mao', 'obra', 'tudo', 'dever', 'contribuir', 'valor', 'acessivel', 'incrivel', 'escritor', 'intelectual', 'colocar', 'cristianismo', 'responsabilidader', 'todo', 'mal', 'universo', 'simplesmente', 'ignor', 'avanco', 'isla', 'radical', 'querer', 'botar', 'hejab', 'burca', 'cabeco', 'cada', 'mulher', 'mundo', 'desejar', 'subjugar', 'mulher', 'extremo', 'ponto', 'pais', 'arabe', 'mulher', 'condenar', 'morte', 'ter', 'ser', 'estuprar', 'ainda',

In [588]:
#Cria função que retorna uma lista sem as palavras repetidas
def remove_repeticao(lista_de_palavras):
    dic = set(lista_de_palavras)
    lista_vocabulario_sem_repeticao = list(dic)
    return lista_vocabulario_sem_repeticao

In [589]:
# Cria uma função que guarda as palavras em pd.Series
#recebe uma lista (devolvida pelo cria vocabulário, preferencialmente)
def cria_pdseries(lista):
    tabela = pd.Series(lista)
    return tabela

#Cria uma função que retorna a frequência absoluta de cada palavra no texto
#recebe uma tabela de pd
def freq_abs(tabela):
    absoluta = tabela.value_counts()
    return absoluta

#Cria uma função que retorna a frequência relativa de cada palavra no texto
def freq_rel(tabela):
    relativa = tabela.value_counts(True)
    return relativa

In [590]:
#Divisão categorias
## Cria uma função que recebe a planilha e o target e 
# retorna o total de palavras e as tabelas de frequencia absoluta e relativa e o vocabulario sem palavras repetidas, nessa ordem
#vou falar que ela recebe a planilha limpa já

def divisao_categorias(planilha_limpa, target):
    #Etapa de divisão de categorias     
    #criou uma nova planilha apenas com as linha com aquele target                
    filtro_target = planilha_limpa.loc[planilha_limpa['Acionável/Direcionável/Não Acionável'] == target]
    vocab_target = cria_vocabulario(filtro_target["Mensagem Limpa"]) #vocabulario daquele target
    df_vocab_target = cria_pdseries(vocab_target)
    freq_rel_target = freq_rel(df_vocab_target)
    freq_abs_target = freq_abs(df_vocab_target)   #frequencia de palavras daquele target
    total_target = freq_abs_target.sum()  #total de palavras daquele target
    #criar vocabulario limpo
    vocab_target_sr = remove_repeticao(vocab_target)   #vocabulario do target sem palavras repetidas
    return total_target, freq_abs_target, freq_rel_target, vocab_target_sr

In [591]:
## Função que recebe a planilha já limpa e retorna:
# o total de palavras com repetição [0] (int)
# a tabela com a frequência absoluta de cada palavra na planilha [1] (Series)
# a tabela com a frequência relativa de cada palavra na planilha [2] (Series)
# a lista com todas as palavras da planilha sem repetição [3] (list)
def divisao_planilhas(planilha_limpa):
    
    coluna_limpa = planilha_limpa["Mensagem Limpa"]                   #separa só a coluna "Mensagem Limpa"
    vocab_planilha = cria_vocabulario(coluna_limpa)             #lista de palavras na coluna "Mensagem Limpa"
    df_vocab_planilha = cria_pdseries(vocab_planilha)       #coloca essa lista em um df
    freq_abs_planilha = freq_abs(df_vocab_planilha)         #calcula a frequência absoluta de cada palavra
    freq_rel_planilha = freq_rel(df_vocab_planilha)         #calcula a frequência relativa de cada palavra
    total_planilha = freq_abs_planilha.sum()                #calcula o total de palavras com repetição
    vocab_planilha_sr = remove_repeticao(vocab_planilha)       #vocabulario sem repeticao (é uma lista)

    return total_planilha, freq_abs_planilha, freq_rel_planilha, vocab_planilha_sr

pequena_limpa = mensagem_limpa(pequena)
# divisao_planilhas(pequena)
pequena_limpa


,Mensagem,Acionável/Direcionável/Não Acionável,Mensagem Limpa
0,"Para quem gosta de poemas simples, esse é o li...",N,gostar poema simples livro ideal autora fazer ...
1,Quando vi o lançamento pensei que finalmente p...,D,ver lancamento pensar finalmente poder livrar ...
2,"É incrível como esses escritores e ""intelectua...",N,incrivel escritor intelectual colocar cristian...
3,"Se desse pra devolver eu devolvia, nao é por e...",N,de esse pra devolver devolvia nao nao pensar i...
4,"Futilidade, inutilidade, desperdício de papel,...",N,futilidade inutilidade desperdicio papel tempo...
5,"O autor tenta mostrar outro lado do livro, bus...",N,autor tentar mostrar outro lado livro buscar r...
6,"Começou boa istoria, no meio um pouco enrolado...",N,comecar boa istor meio pouco enrolar embromar ...
7,"Bem, eu nem sei por onde começar. A história é...",N,bem saber onde comecar historio representacao ...
8,"Menos do menos, não mais do mesmo. As dicas da...",N,menos menos nao dica obra sao lugarcomum nao t...
9,Meu box veio todo detonado. Fiquei triste porq...,A,box vir todo detonar fiquei triste porque nunc...


In [592]:
#Função de Suavização de LaPlace

#vou criar um dicionário com todas as palavras e seus valores de 
# probabilidade dado target já suavizados
# aí vai ficar mais rapido de encontrar os valores
# do que ter que suavizar pra cada palavra toda vez
# que chamar o loop

#APLICAMOS A SUAVIZAÇÃO PARA TODAS AS PALAVRAS DA BASE DE TREINO PARA QUE EXISTISSE AO MENOS UMA PROBABILIDADE DELA 
#ESTAR EM CADA UMA DAS CLASSES DE TARGET
#DEPOIS, CASO A PALAVRA DA BASE DE TESTE NAO EXISTA NA MINHA BASE DE TREINO, EU APLICO UMA NOVA SUAVIZAÇÃO PARA ELA
    
def suaviza_target (planilha_limpa, target):

    coluna_limpa = planilha_limpa["Mensagem Limpa"]
    
    #extraindo dados da planilha
    dados_target = divisao_categorias(planilha_limpa, target)
    frequencias_absolutas = dados_target[1]
    qtdd_palavras = dados_target[0]                  #quantidade de palavras no target informado
    qtdd_palavras_sem_repeticao = len(dados_target[3]) #quantidade de palavras sem repetição no target informado
    
    dic = {}   #dicionario com chave de palavra e valor probabilidade condicional
    
    alfa = 0.01
    
    for frase in coluna_limpa:
        frase = frase.split()
        for palavra in frase:
            if palavra not in dic:
                if palavra in frequencias_absolutas:
                    freq_abs_palavra = frequencias_absolutas[palavra]
                    Prob_palavra_dado_target = (freq_abs_palavra + alfa)/(qtdd_palavras + alfa*qtdd_palavras_sem_repeticao)
                    dic[palavra] = Prob_palavra_dado_target
                else:
                    freq_abs_palavra = 0
                    Prob_palavra_dado_target = (freq_abs_palavra + alfa)/(qtdd_palavras + alfa*qtdd_palavras_sem_repeticao)
                    dic[palavra]= Prob_palavra_dado_target

    return dic

In [593]:
# Calculando probabilidade da frase dado target
# recebe uma frase qualquer (precisa estar limpa)
# def frase_dado_target(frase, target, planilha_limpa):
    
#     prob_frase_dado_classe = 1
#     frase =  frase.split()
#     dic_probabilidades_por_classe = suaviza_target(planilha_limpa, target)   #dicionario com as palavras e suas probabilidades condicionais
#     for palavra in frase:
#         if palavra in dic_probabilidades_por_classe:
#             prob_palavra_dado_classe = dic_probabilidades_por_classe[palavra]
#             prob_frase_dado_classe *= prob_palavra_dado_classe
#         else:
#             freq_abs_palavra = 0
#             dados_target = divisao_categorias(planilha_limpa, target)
#             qtdd_palavras = dados_target[0]                  #quantidade de palavras no target informado
#             qtdd_palavras_sem_repeticao = len(dados_target[3]) #quant  
#             prob_palavra_dado_classe = (freq_abs_palavra + 1)/(qtdd_palavras + qtdd_palavras_sem_repeticao)
#             prob_frase_dado_classe *= prob_palavra_dado_classe
#     return prob_frase_dado_classe

# frase = "Eu odiei esse livro, achei realmente uma merda !!? "
# frase_dado_target(frase, "N", planilha_limpa)

def frase_dado_target(frase, target, planilha_limpa):
    
    prob_frase_dado_classe = 1
    frase =  frase.split()
    dic_probabilidades_por_classe = suaviza_target(planilha_limpa, target)   #dicionario com as palavras e suas probabilidades condicionais
    alfa = 0.01
    for palavra in frase:
        if palavra in dic_probabilidades_por_classe:
            prob_palavra_dado_classe = dic_probabilidades_por_classe[palavra]
            prob_frase_dado_classe *= prob_palavra_dado_classe
        else:
            freq_abs_palavra = 0
            dados_target = divisao_categorias(planilha_limpa, target)
            qtdd_palavras = dados_target[0]                  #quantidade de palavras no target informado
            qtdd_palavras_sem_repeticao = len(dados_target[3]) #quant  
            prob_palavra_dado_classe = (freq_abs_palavra + alfa)/(qtdd_palavras + alfa*qtdd_palavras_sem_repeticao)
            prob_frase_dado_classe *= prob_palavra_dado_classe
    return prob_frase_dado_classe



    # def frase_dado_target(frase, target, planilha_limpa):
        
    #     prob_frase_dado_classe = 1
    #     frase =  frase.split()
    #     dic_probabilidades_por_classe = suaviza_target(planilha_limpa, target)   #dicionario com as palavras e suas probabilidades condicionais
    #     alfa = 0.5
    #     for palavra in frase:
    #         if palavra in dic_probabilidades_por_classe:
    #             prob_palavra_dado_classe = dic_probabilidades_por_classe[palavra]
    #             prob_frase_dado_classe *= prob_palavra_dado_classe
    #         else:
    #             prob_frase_dado_classe *= 1
    #     return prob_frase_dado_classe

In [594]:
#Função de classificação das frases (quanto a acionável, não acionável e direcionável)
#recebe a função já limpa
def classificador(frase, planilha):
    
    #Extraindo as informações
    total_planilha_N = divisao_categorias(planilha, "N")[0]
    total_planilha_D = divisao_categorias(planilha, "D")[0]
    total_planilha_A = divisao_categorias(planilha, "A")[0]
    total_planilha = divisao_planilhas(planilha)[0]
    
    #Cálculo das probabilidades
    P_frase_dado_A = frase_dado_target(frase, "A", planilha)
    P_frase_dado_D = frase_dado_target(frase, "D", planilha)
    P_frase_dado_N = frase_dado_target(frase, "N", planilha)
    P_N = total_planilha_N/total_planilha         #probabilidade de estar na categoria N
    P_D = total_planilha_D/total_planilha         #probabilidade de estar na categoria D
    P_A = total_planilha_A/total_planilha         #probabilidade de estar na categoria A
    
    P_A_dado_frase = P_frase_dado_A*P_A
    P_N_dado_frase = P_frase_dado_N*P_N
    P_D_dado_frase = P_frase_dado_D*P_D
        
    #Classificação
    if P_A_dado_frase >= P_D_dado_frase and P_A_dado_frase >= P_N_dado_frase:    #na classificação manual quando impatava a prioridade era acionável
        return "A"
    elif P_D_dado_frase > P_N_dado_frase and P_D_dado_frase > P_A_dado_frase:
        return "D"
    elif P_N_dado_frase > P_D_dado_frase and P_N_dado_frase > P_A_dado_frase:
        return "N"
    else:
        return "Houve um impasse"    

In [568]:
# #TENTATIVA DE OTIMIZAR (Toda comentada)
# #cria um dic com probabilidade de frase dado target para todas as frases

# def dic_prob_frase_dado_target(planilha_limpa, target):
#     dic_prob_frase_dado_target = {}
#     #cria uma planilha com o filtro aplicado
#     for frase in planilha_limpa["Mensagem"]:
#         prob_frase_dado_target = frase_dado_target(frase, target, planilha_limpa)
#         if frase not in dic_prob_frase_dado_target:
#             dic_prob_frase_dado_target[frase] = prob_frase_dado_target
#     return dic_prob_frase_dado_target

# def classifica_por_dic(planilha_limpa):
#     dic_prob_frase_dado_N = dic_prob_frase_dado_target(planilha_limpa, "N")
#     dic_prob_frase_dado_A = dic_prob_frase_dado_target(planilha_limpa, "A")
#     dic_prob_frase_dado_D = dic_prob_frase_dado_target(planilha_limpa, "D")
#     total_planilha_N = divisao_categorias(planilha_limpa, "N")[0]
#     total_planilha_D = divisao_categorias(planilha_limpa, "D")[0]
#     total_planilha_A = divisao_categorias(planilha_limpa, "A")[0]
#     total_planilha = divisao_planilhas(planilha_limpa)[0]
     

# def classificador(frase, planilha_limpa, dic_prob_frase_dado_A, dic_prob_frase_dado_D, dic_prob_frase_dado_N):
#     total_train_N = divisao_categorias(planilha_limpa, "N")[0]
#     total_train_D = divisao_categorias(planilha_limpa, "D")[0]
#     total_train_A = divisao_categorias(planilha_limpa, "A")[0]
#     total_train = divisao_planilhas(train)[0]
#     if frase in dic_prob_frase_dado_A:
#         P_A_dado_frase = dic_prob_frase_dado_A[frase]*

# dic_prob_frase_dado_N = dic_prob_frase_dado_target(planilha_limpa, "N")
# dic_prob_frase_dado_A = dic_prob_frase_dado_target(planilha_limpa, "A")
# dic_prob_frase_dado_D = dic_prob_frase_dado_target(planilha_limpa, "D")
    
    
#     dic = dic_prob_frase_dado_target(planilha_limpa, target)
    

# print(dic_prob_frase_dado_N)
# print(dic_prob_frase_dado_A)
# print(dic_prob_frase_dado_D)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [595]:
## Fazer meu classificador classificar todas as frases da planilha de teste
#recebe a planilha inteira, não precisa realizar nenhuma limpeza
def classifica_planilha(planilha_teste, planilha_treino):
    planilha_teste_limpa = mensagem_limpa(planilha_teste)
    planilha_treino_limpa = mensagem_limpa(planilha_treino)
    avaliacoes_limpas = planilha_teste_limpa["Mensagem Limpa"]
    #fez uma cópia da planilha para nao alterar a antiga
    planilha_nova = planilha_teste_limpa.copy()
    # Criar uma lista para armazenar os resultados do classificador
    resultados = []
    for frase in avaliacoes_limpas:
        resultado = classificador(frase, planilha_treino_limpa)
        resultados.append(resultado)
    # Atribuir a lista de resultados à coluna 'Classificador Automático'
    planilha_nova['Classificador Automático'] = resultados
    return planilha_nova

# planilha_classificada = classifica_planilha(test, train)
# planilha_classificada



In [596]:
#realizando o cruzamento de dados da planilha

pd.crosstab(planilha_classificada['Classificador Automático'], planilha_classificada['Acionável/Direcionável/Não Acionável'],normalize=True, margins=True)

Acionável/Direcionável/Não Acionável,A,D,N,All
Classificador Automático,,,,
A,0.132,0.068,0.300,0.500
D,0.012,0.124,0.128,0.264
N,0.000,0.000,0.236,0.236
All,0.144,0.192,0.664,1.000


In [597]:

#verificando a acurácia

def verifica_performance(planilha_teste, planilha_treino):
    planilha_classificada = classifica_planilha(planilha_teste, planilha_treino)

    count_comparativo = (planilha_classificada['Acionável/Direcionável/Não Acionável'] == planilha_classificada['Classificador Automático']).sum() # Conta quantos valores nas duas são correspondentes
    total_elementos = len(planilha_classificada) #Total de linhas

    porcentagem_concordancia = (count_comparativo / total_elementos) * 100
    
    return porcentagem_concordancia, count_comparativo, total_elementos

print(verifica_performance(test, train))



(83.6, 209, 250)


In [598]:
#nessa suavizamos todo o dicionario

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

Dica: apresentar um grafico com testes mostrando quais limpezas melhoraram a acurácia do nosso classificador (lemmatization, stopwords, etc)

REtirar algumas palavras para ver se melhora a qualidade do classificador (exemplo palavra não)